In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir("/content/drive/MyDrive/NLP_Projects/NLP_Transformer")

In [ ]:
!dir

#Step 1:Import Dependences

In [ ]:
import numpy as np
import math
import re
import time
from google.colab import drive

In [ ]:
try:
    %tensorflow_version 2.x
except:
    pass
import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_datasets as tfds

##Load Dataset

In [ ]:
with open("/content/drive/MyDrive/NLP_Projects/NLP_Transformer/europarl-v7.fr-en.en",
          mode='r',
          encoding="utf-8") as f:
          europarl_en = f.read()
with open("/content/drive/MyDrive/NLP_Projects/NLP_Transformer/europarl-v7.fr-en.fr",
          mode='r',
          encoding="utf-8") as f:
          europarl_fr = f.read()
with open("/content/drive/MyDrive/NLP_Projects/NLP_Transformer/nonbreaking_prefix.en",
          mode='r',
          encoding="utf-8") as f:
          nonbreaking_prefix_en = f.read()
with open("/content/drive/MyDrive/NLP_Projects/NLP_Transformer/nonbreaking_prefix.fr",
          mode='r',
          encoding="utf-8") as f:
          nonbreaking_prefix_fr = f.read()

In [ ]:
europarl_fr[:50]

In [ ]:
europarl_en[:50]

#Step 2:Data Preprocessing

##Cleaning Data

Getting the nonbreaking_prefix_en as a clean list of words with a point at the end so it is easier to use.

In [ ]:
nonbreaking_prefix_en = nonbreaking_prefix_en.split("\n")
nonbreaking_prefix_en = [' ' + pref + '.' for pref in nonbreaking_prefix_en]

In [ ]:
nonbreaking_prefix_fr = nonbreaking_prefix_fr.split("\n")
nonbreaking_prefix_fr = [' ' + pref + '.' for pref in nonbreaking_prefix_fr]

In [ ]:
nonbreaking_prefix_en

In [ ]:
nonbreaking_prefix_fr

we will need each word and other symbol that we want to keep to be in a lower case and separeated by spaces so we can "tokenize" them.

In [ ]:
corpus_en = europarl_en
for prefix in nonbreaking_prefix_en:
    corpus_en = corpus_en.replace(prefix, prefix + "###")
corpus_en = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".###", corpus_en)
corpus_en = re.sub(r"\.###", '', corpus_en)
corpus_en = re.sub(r" +", ' ', corpus_en)
corpus_en = corpus_en.split("\n")

corpus_fr = europarl_fr
for prefix in nonbreaking_prefix_fr:
    corpus_fr = corpus_fr.replace(prefix, prefix + "###")
corpus_fr = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".###", corpus_fr)
corpus_fr = re.sub(r"\.###", '',corpus_fr)
corpus_fr = re.sub(r" +", ' ', corpus_fr)
corpus_fr = corpus_fr.split("\n")

##Tokenizing

In [ ]:
!pip install tensorflow-text

In [ ]:
!pip install tensorflow-datasets

In [ ]:
import tensorflow_text as text
import tensorflow_datasets as tfds

In [ ]:
import tensorflow_datasets as tfds

# Create tokenizers
tokenizer_en = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(corpus_en, target_vocab_size=2**13)
tokenizer_fr = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(corpus_fr, target_vocab_size=2**13)

In [ ]:
VOCAB_SIZE_EN = tokenizer_en.vocab_size + 2
VOCAB_SIZE_FR = tokenizer_fr.vocab_size + 2

In [ ]:
inputs = [[VOCAB_SIZE_EN-2] + tokenizer_en.encode(sentence) + [VOCAB_SIZE_EN-1]
          for sentence in corpus_en]
outputs = [[VOCAB_SIZE_FR-2] + tokenizer_fr.encode(sentence) + [VOCAB_SIZE_FR-1]
           for sentence in corpus_fr]

##Removing TOO long sentences

In this we are removing sentences more then 20 length, it will help us to compute faster.If we are having enough ram and GPU than this setp is not for you

In [ ]:
MAX_LENGTH = 20
idx_to_remove = [count for count, sent in enumerate(inputs) if len(sent) > MAX_LENGTH]
for idx in reversed(idx_to_remove):
    del inputs[idx]
    del outputs[idx]

idx_to_remove = [count for count, sent in enumerate(outputs) if len(sent) > MAX_LENGTH]
for idx in reversed(idx_to_remove):
    del inputs[idx]
    del outputs[idx]

##Inputs/Outputs Creation

As we train with batches, we need each input to have the same length. We pad with the appropriate token,and will make sure this padding token dosen't interfere with our training later.

In [ ]:
MAX_LENGTH = 20

# Assuming inputs and outputs are sequences to be padded
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                       value=0,
                                                       padding="post",
                                                       maxlen=MAX_LENGTH)
outputs = tf.keras.preprocessing.sequence.pad_sequences(outputs,
                                                        value=0,
                                                        padding="post",
                                                        maxlen=MAX_LENGTH)


In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))

In [ ]:
dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dateset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

#Stage 3: Model building

##Embedding

Positional encoding formulae:

PE(pos,2i) = sin(pos/100002i/dmodel)

PE(pos,2i+1) = cos(pos/100002i/dmodel)

In [ ]:
import numpy as np

class PositionalEncoding(layers.Layer):
    def __init__(self):
        super(PositionalEncoding, self).__init__()

    def get_angles(self, pos, i, d_model):
        angles = 1/np.power(10000., (2*(i//2))/np.float32(d_model))
        return pos * angles


    def call(self, inputs):
        seq_length = inputs.shape.as_list()[-2]
        d_model = inputs.shape.as_list()[-1]
        angles = self.get_angles(np.arange(seq_length)[:, np.newaxis],  # Check this line
                                 np.arange(d_model)[np.newaxis, :],
                                 d_model)
        angles[:, 0::2] = np.sin(angles[:, 0::2])
        angles[:, 1::2] = np.cos(angles[:, 1::2])
        pos_encoding = angles[np.newaxis, ...]
        return inputs + tf.cast(pos_encoding, tf.float32)


##Attention

###Attention computation

Attention(Q,K,V)=softmax ((Q * K ^ T)/(sqrt(d_{k}))) * V

In [ ]:
def scaled_dot_product_attention(queries, keys, values, mask):
    product = tf.matmul(queries, keys, transpose_b=True)

    keys_dim = tf.cast(tf.shape[keys][-1], tf.float32)
    scaled_product = product / tf.math.sqrt(keys_dim)

    if mask is not None:
        scaled_product += (mask * -1e9)

    attention = tf.matmul(tf.nn.softmax(scaled_product, axis=-1), values)

    return attention


##Multi-head attention sublayer

In [ ]:
class MultiHeadAttention(layers.Layer):

    def _init__(self, nb_proj):
        super(MultiHeadAttention, self)._init_()
        self.nb_proj = nb_proj

    def build(self, input_shape):
        self.d_model = input_shape[-1]
        assert self.d_model & self.nb_proj == 0

        self.d_proj = self.d_model // self.nb_proj

        self.query_lin = layers.Dense(units=self.d_model)
        self.key_lin = layers.Dense(units=self.d_model)
        self.value_lin = layers.Dense(units=self.d_model)

        self.final_lin = layers.Dense(units=self.d_model)

    def split_proj(self, inputs, batch_size):
        shape = (batch_size,
                 -1,
                 self.nb_proj,
                 self.d_proj)
        splited_inputs = tf.reashpe(inputs, shape=shape)
        return tf.transpose(splited_inputs, perm=[0, 2, 1, 3])

    def call(self, queries, keys, values, mask):
        batch_size = tf.shape(queries)[0]

        queries = self.query_lin(queries)
        keys = self.key_list(keys)
        values = self.value_list(values)

        queries = self.split_proj(queries, batch_size)
        keys = self.split_proj(keys, batch_size)
        values = self.split_proj(values, batch_size)

        attention = scaled_dot_product_attention(queries, keys, values, mask)

        attention = tf.transpose(attention, param=[0, 2, 1, 3])

        concat_attention = tf.reshape(attention,
                                      shape=(batch_size, -1,self.d_model))

        outputs = self.final_lin(concat_attention)

        return outputs


##Encoder

In [ ]:
class EncoderLayer(layers.Layer):

    def __init__(self, FFN_units, nb_proj, dropout):
         super(EncoderLayer, self).__init__()
         self.FFN_units = FFN_units
         self.nb_proj = nb_proj
         self.dropout = dropout

    def build(self, input_shape):
        self.d_model = input_shape[-1]

        self.multi_head_attention = MultiHeadAttention(self.nb_proj)
        self.dropout_1 = layers.Dropout(rate=self.dropout)
        self.norm_1 = layers.LayersNormalization(epsilon=1e-6)

        self.dense_1 = layers.Dense(units=self.FFN_units, activation="relu")
        self.dense_2 = layers.Dense(units=self.d_model)
        self.dropout_2 = layers.Dropout(rate=self.dropout)
        self.norm_2 = layers.LayersNormalization(epsilon=1e-6)

    def call(self, inputs, mask, training):
        attention = self.multi_head_attention(inputs,
                                              inputs,
                                              inputs,
                                              mask)
        attention = self.dropout_1(attention, training=training)
        attention = self.norm_1(attention + inputs)

        outputs = self.dense_1(attention)
        outputs = self.dense_2(outputs)
        outputs = self.dropout_2(outputs)
        outputs = self.norm_2(outputs)

        return outputs

In [ ]:
class Encoder(layers.Layer):

    def _init_(self,
               nb_layers,
               FFN_units,
               nb_proj,
               dropout,
               vocab_size,
               d_model,
               name="encoder"):
        super(Encoder, self)._init_(name=name)
        self.nb_layers = nb_layers
        self.d_model = d_model

        self.embedding = layers.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding()
        self.dropout = layers.Dropout(rate=dropout)
        self.enc_layers = [EncoderLayer(FFN_units,
                                        nb_proj,
                                        dropout)
                            for _ in range(nb_layers)]

    def call(self, inputs, mask, training):
        outputs = self.embedding(inputs)
        outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        outputs = self.pos_encoding(outputs)
        outputs = self.dropout(outputs, training)

        for i in range(self.nb_layers):
            outputs = self.enc_layers[i](outputs, mask, training)

        return outputs

##Decoder

In [ ]:
class DecoderLayer(layers.Layer):

    def __init__(self,
               nb_layers,
               FFN_units,
               nb_proj,
               dropout,
               vocab_size,
               d_model,
               name="encoder"):
        super(Encoder, self).__init__(name=name)
        self.nb_layers = nb_layers
        self.d_model = d_model

        self.embedding = layers.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding()
        self.dropout = layers.Dropout(rate=dropout)
        self.enc_layers = [EncoderLayer(FFN_units,
                                        nb_proj,
                                        dropout)
                            for _ in range(nb_layers)]

    def call(self, inputs, mask, training):
        outputs = self.embedding(inputs)
        outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        outputs = self.pos_encoding(outputs)
        outputs = self.dropout(outputs, training)

        for i in range(self.nb_layers):
            outputs = self.enc_layers[i](outputs, mask, training)

        return outputs


In [ ]:
class Decoder(layers.Layer):

    def __init__(self,
               nb_layers,
               FFN_units,
               nb_proj,
               dropout,
               vocab_size,
               d_model,
               name="decoder"):
        super(Decoder, self).__init__(name=name)
        self.nb_layers = nb_layers
        self.d_model = d_model

        self.embedding = layers.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding()
        self.dropout = layers.Dropout(rate=dropout)

        self.dec_layers = [DecoderLayer(FFN_units,
                                        nb_proj,
                                        dropout)
                            for _ in range(nb_layers)]

    def call(self, inputs, enc_outputs,mask_1, mask_2, training):
        outputs = self.embedding(inputs)
        outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        outputs = self.pos_encoding(outputs)
        outputs = self.dropout(outputs, training)

        for i in range(self.nb_layers):
            outputs = self.dec_layers[i](outputs,
                                         enc_outputs,
                                         mask_1,
                                         mask_2,
                                         training)

        return outputs


##Transformer

In [ ]:
class Transformer(tf.keras.Model):

    def __init__(self,
                 vocab_size_enc,
                 vocab_size_dec,
                 d_model,
                 nb_layers,
                 FFN_units,
                 nb_proj,
                 dropout,
                 name="transformer"):
         super(Transformer, self).__init__(name=name)

         self.encoder = Encoder(nb_layers,
                                FFN_units,
                                nb_proj,
                                dropout,
                                vocab_size_enc,
                                d_model)
         self.decoder = Decoder(nb_layers,
                                FFN_units,
                                nb_proj,
                                dropout,
                                vocab_size_dec,
                                d_model)
         self.last_linear = layers.Dense(units=vocab_size_dec)

    def create_padding_mask(self, seq):  # seq: (batch_size, seq_length)
        mask = tf.cast(tf.math.equal(seq, 0), tf.float32)
        return mask[:, tf.newaxis, tf.newaxis, :]

    def create_look_ahead_mask(self, seq):
        seq_len = tf.shape(seq)[1]
        look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        return look_ahead_mask

    def call(self, enc_inputs, dec_inputs, training):
        enc_mask = self.create_padding_mask(enc_inputs)
        dec_mask_1 = tf.maximum(
            self.create_padding_mask(dec_inputs),
            self.create_look_ahead_mask(dec_inputs)
        )
        dec_mask_2 = self.create_padding_mask(enc_inputs)

        enc_outputs = self.encoder(enc_inputs, enc_mask, training)
        dec_outputs = self.decoder(dec_inputs,
                                   enc_outputs,
                                   dec_mask_1,
                                   dec_mask_2,
                                   training)

        outputs = self.last_linear(dec_outputs)

        return outputs


#Training

In [ ]:
tf.keras.backend.clear_session()

#Hyper Parameters    #Article Hyper Parameters values
D_MODEL = 128        #512
NB_LAYERS = 4        #6
FFN_UNITS = 512      #2048
NB_PROJ = 8          #8
DROPOUT = 0.1       #0.1

transformer = Transformer(vocab_size_enc=VOCAB_SIZE_EN,
                          vocab_size_dec=VOCAB_SIZE_FR,
                          d_model=D_MODEL,
                          nb_layers=NB_LAYERS,
                          FFN_units=FFN_UNITS,
                          nb_proj=NB_PROJ,
                          dropout=DROPOUT,
                          name="transformer")

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction="none")

def loss_function(target, pred):
    mask = tf.math.logical_not(tf.math.equal(target, 0))
    loss_ = loss_object(target, pred)

    mask = tf.cast(mask, dtypes=loss_.dtypes)
    loss_ *= mask

    return tf.reduce_mean(loss_)

train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=4000):
         super(CustomSchedule, self).__init__()

         self.d_model = tf.cast(d_model, tf.float32)
         self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)  # Cast step to float32
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)



learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                   beta_1=0.9,
                                   beta_2=0.98,
                                   epsilon=1e-9)

checkpoint_path = "/content/drive/MyDrive/NLP_Projects/NLP_Transformer/ckpt"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

In [ ]:
import numpy as np

In [ ]:
EPOCHS = 10
for epoch in range(EPOCHS):  # Use "in" instead of "range"
    print("Start of epoch {}".format(epoch+1))
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()

    for (batch, (enc_inputs, targets)) in enumerate(dataset):
        dec_inputs = targets[:, :-1]
        dec_outputs_real = targets[:, 1:]
        with tf.GradientTape() as tape:
            predictions = transformer(enc_inputs, dec_inputs, True)
            loss = loss_function(dec_outputs_real, predictions)

        gradients = tape.gradient(loss, transformer.trainable_variables)
        optimizer.apply_gradient(zip(gradients, transformer.trainable_variables))

        train_loss(loss)
        train_accuracy(dec_outputs_real, predictions)

        if batch % 50 == 0:
            print("Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}".format(
                epoch+1, batch, train_loss.result(), train_accuracy.result()
            ))

    ckpt_save_path = ckpt_manager.save()
    print("Saving checkpoints for epoch {} at {}".format(epoch+1,
                                                         ckpt_save_path))
    print("Time taken for 1 epoch: {} secs\n".format(time.time() - start))


#Evaluating

In [ ]:
def evaluate(inp_sentence):
    inp_sentence = \
        [VOCAB_SIZE_EN-2] + tokenizer_en.encoder(inp_sentence) + [VOCAB_SIZE_EN-1]
    enc_inputs = tf.expand_dims(inp_sentence, axis=0)

    output = tf.expand_dims([VOCAB_SIZE_FR-2], axis=0)

    for _ in range[MAX_LENGTH]:
        predictions = transformer(enc_input, output, False) # (1, seq_length, vocab_size_fr)

        prediction = predictions[:, -1:, :]

        prediction_id = tf.cast(tf.argmax(prediction, axis=-1), tf.int32)

        if prediction_id == VOCAB_SIZE_FR-1:
            return tf.squeeze(output, axis=0)

        output = tf.concat([output, prediction_id], axis=-1)

    return tf.squeeze(output, axis=0)

In [ ]:
def translate(sentence):
    output = evaluate(sentence).numpy()

    predicted_sentence = tokenizer_fr.decoder(
        [i for i in output if i < VOCAB_SIZE_FR-2]
    )

    print("Input: {}.format(sentence)")
    print("Predicted translation: {}".format(predicted_sentence))

In [ ]:
translate("This is a problem we have to solve.")